# Exercise: Heterogenous Plugin and the DevCloud

Now that you've learned about the **Heterogenous plugin**, you will have the opportunity to load a model on all three hardware devices, CPU, GPU, and FPGA, using 3 device combinations and run inference on an image.

In this exercise, you will do the following:
1. Write a Python script to load a model and run inference 100 times on a device on Intel's DevCloud.
    * Calculate the time it takes to load the model.
    * Calculate the time it takes to run inference 100 times.
2. Write a shell script to submit a job to Intel's DevCloud.
3. Submit a job using `qsub` on an **IEI Tank-870** edge node using `HETERO`, run `liveQStat` to view the status of your submitted jobs, and then retrieve and view the results from your job.
    * One job using `FPGA/CPU` as the device.
    * One job using `GPU/CPU` as the device.
    * One job using `FPGA/GPU/CPU` as the device.
4. Plot and compare the results using bar graphs with `matplotlib` for the following metrics:
    * Model Loading Time
    * Inference Time
    * Frames Per Second (FPS)

Click the **Exercise Overview** button below for a demonstration.

<span class="graffiti-highlight graffiti-id_z8bfs11-id_d97ox8f"><i></i><button>Exercise Overview</button></span>

#### IMPORTANT: Set up paths so we can run Dev Cloud utilities
You *must* run this every time you enter a Workspace session.

In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


## The Model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise.

Remember to use the appropriate model precisions for each device:

* FPGA - `FP16`
* GPU - `FP16`
* CPU - It is prefered to use `FP32`, but we have to use `FP16` since **GPU** and **FPGA** use `FP16`

The model has already been downloaded for you in the `/data/models/intel` directory on Intel's DevCloud.

We will be running inference on an image of a car. The path to the image is `/data/resources/car.png`.

# Step 1: Creating a Python Script

The first step is to create a Python script that you can use to load the model and perform inference. We'll use the `%%writefile` magic to create a Python file called `inference_on_device.py`. In the next cell, you will need to complete the `TODO` items for this Python script.

`TODO` items:

1. Load the model

2. Get the name of the input node

3. Prepare the model for inference (create an input dictionary)

4. Run inference 100 times in a loop

If you get stuck, you can click on the **Show Solution** button below for a walkthrough with the solution code.

In [ ]:
%%writefile inference_on_device.py

import time
import numpy as np
import cv2
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
import argparse

def main(args):
    model=args.model_path
    model_weights=model+'.bin'
    model_structure=model+'.xml'
    
    start=time.time()
    
    # TODO: Load the model
    
    load_time=time.time()-start
    print(f"Time taken to load model = {load_time} seconds")
    
    # Get the name of the input node
    
    # Reading and Preprocessing Image
    input_img=cv2.imread('car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)

    # TODO: Prepare the model for inference (create input dict etc.)
    
    start=time.time()
    for _ in range(100):
        # TODO: Run Inference in a Loop
    
    inference_time=time.time()-start
    fps=100/inference_time
    
    print(f"Time Taken to run 100 Inference is = {inference_time} seconds")
    
    with open(f"/output/{args.path}.txt", "w") as f:
        f.write(str(load_time)+'\n')
        f.write(str(inference_time)+'\n')
        f.write(str(fps)+'\n')

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    parser.add_argument('--device', default=None)
    parser.add_argument('--path', default=None)
    
    args=parser.parse_args() 
    main(args)

<span class="graffiti-highlight graffiti-id_1rnmf5g-id_nmeqj1a"><i></i><button>Show Solution</button></span>

## Step 2: Creating a Job Submission Script

To submit a job to the DevCloud, you'll need to create a shell script. Similar to the Python script above, we'll use the `%%writefile` magic command to create a shell script called `inference_model_job.sh`. In the next cell, you will need to complete the `TODO` items for this shell script.

`TODO` items:
1. Create three variables:
    * `DEVICE` - Assign the value as the first argument passed into the shell script.
    * `MODELPATH` - Assign the value as the second argument passed into the shell script.
    * `SAVEPATH` - Assign the value as the third argument passed into the shell script.
2. Call the Python script using the three variable values as the command line argument

If you get stuck, you can click on the **Show Solution** button below for a walkthrough with the solution code.

In [ ]:
%%writefile inference_model_job.sh
#!/bin/bash

exec 1>/output/stdout.log 2>/output/stderr.log

mkdir -p /output

# TODO: Create DEVICE variable
# TODO: Create MODELPATH variable
# TODO: Create SAVEPATH variable

if echo "$DEVICE" | grep -q "FPGA"; then # if device passed in is FPGA, load bitstream to program FPGA
    export AOCL_BOARD_PACKAGE_ROOT=/opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/2020-2_PL2_FP16_MobileNet_Clamp.aocx
    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi

# TODO: Call the Python script

cd /output

tar zcvf output.tgz * # compresses all files in the current directory (output)

<span class="graffiti-highlight graffiti-id_f1nbmn9-id_ia7yjlq"><i></i><button>Show Solution</button></span>

## Step 3: Submitting a Job to Intel's DevCloud

In the next three sub-steps, you will write your `!qsub` commands to submit your jobs to Intel's DevCloud to load your model and run inference on the **IEI Tank-870** edge node with an **Intel Core i5** CPU and an **Intel Arria 10 FPGA**. You will use the **HETERO** device plugin to run inference on three device combinations.

Your `!qsub` command should take the following flags and arguments:
1. The first argument should be the shell script filename
2. `-d` flag - This argument should be `.`
3. `-l` flag - This argument should request an edge node with an **IEI Tank-870**. The default quantity is 1, so the **1** after `nodes` is optional. 
    * **Intel Core i5 6500TE** for your `CPU`.
    * **Intel HD Graphics 530** for your `IGPU`.
    * **Intel Arria 10** for your `FPGA`.

To get the queue labels for these devices, you can go to [this link](https://devcloud.intel.com/edge/get_started/devcloud/)

4. `-F` flag - This argument should contain the three values to assign to the variables of the shell script:
    * **DEVICE** - Device type for the job: You will have to use `HETERO` with three different combinations of `CPU`,`GPU` or `FPGA`.
        - `FPGA,CPU`
        - `GPU,CPU`
        - `FPGA,GPU,CPU`
    * **MODELPATH** - Full path to the model for the job. As a reminder, the model is located in `/data/models/intel`.
    * **SAVEPATH** - Name of the file you want to save the performance metrics as. These should be named as the following:
        - `fpga_cpu_stats` for the `FPGA/CPU` job
        - `gpu_cpu_stats` for the `GPU/CPU` job
        - `fpga_gpu_cpu_stats` for the `FPGA/GPU/CPU` job

**Note**: There is an optional flag, `-N`, you may see in a few exercises. This is an argument that only works on Intel's DevCloud that allows you to name your job submission. This argument doesn't work in Udacity's workspace integration with Intel's DevCloud.

## Step 3a: Running on the FPGA and CPU

In the cell below, write the qsub command that will submit your job to both the FPGA and CPU.

If you get stuck, you can click on the **Show Solution** button below for a walkthrough with the solution code.

In [ ]:
fpga_cpu_job = # TODO: Write qsub command
print(fpga_cpu_job[0])

<span class="graffiti-highlight graffiti-id_cvp3lyi-id_chmeh50"><i></i><button>Show Solution</button></span>

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job. The cell is locked until this finishes polling 10 times or you can interrupt the kernel to stop it by pressing the stop button at the top: ![stop button](assets/interrupt_kernel.png)

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

###### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results

get_results.getResults(fpga_cpu_job[0], filename="output.tgz", blocking=True)

###### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

###### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

## Step 3a: Running on the GPU and CPU

In the cell below, write the qsub command that will submit your job to both the GPU and CPU.

If you get stuck, you can click on the **Show Solution** button below for a walkthrough with the solution code.

In [ ]:
gpu_cpu_job = # TODO: Write qsub command
print(gpu_cpu_job[0])

<span class="graffiti-highlight graffiti-id_7k34s6u-id_022l4bj"><i></i><button>Show Solution</button></span>

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job. The cell is locked until this finishes polling 10 times or you can interrupt the kernel to stop it by pressing the stop button at the top: ![stop button](assets/interrupt_kernel.png)

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

###### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results

get_results.getResults(gpu_cpu_job[0], filename="output.tgz", blocking=True)

###### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

###### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

## Step 3c: Running on the FPGA, GPU, and CPU

In the cell below, write the qsub command that will submit your job to all three devices, FPGA, GPU, and CPU.

If you get stuck, you can click on the **Show Solution** button below for a walkthrough with the solution code.

In [ ]:
fpga_gpu_cpu_job = # TODO: Write qsub command
print(fpga_gpu_cpu_job[0])

<span class="graffiti-highlight graffiti-id_mxh5ozv-id_qicoukm"><i></i><button>Show Solution</button></span>

### Check Job Status

To check on the job that was submitted, use `liveQStat` to check the status of the job. The cell is locked until this finishes polling 10 times or you can interrupt the kernel to stop it by pressing the stop button at the top: ![stop button](assets/interrupt_kernel.png)

Column `S` shows the state of your running jobs.

For example:
- If `JOB ID`is in Q state, it is in the queue waiting for available resources.
- If `JOB ID` is in R state, it is running.

In [ ]:
import liveQStat
liveQStat.liveQStat()

###### Get Results

Run the next cell to retrieve your job's results.

In [ ]:
import get_results

get_results.getResults(fpga_gpu_cpu_job[0], filename="output.tgz", blocking=True)

###### Unpack your output files and view stdout.log

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log

###### View stderr.log
This can be used for debugging

In [ ]:
!cat stderr.log

## Step 4: Plot and Compare Results

Run the cells below to plot and compare the results.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(labels, data, title, label):
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_ylabel(label)
    ax.set_title(title)
    ax.bar(labels, data)
    
def read_files(paths, labels):
    load_time=[]
    inference_time=[]
    fps=[]
    
    for path in paths:
        if os.path.isfile(path):
            f=open(path, 'r')
            load_time.append(float(f.readline()))
            inference_time.append(float(f.readline()))
            fps.append(float(f.readline()))

    plot(labels, load_time, 'Model Load Time', 'seconds')
    plot(labels, inference_time, 'Inference Time', 'seconds')
    plot(labels, fps, 'Frames per Second', 'Frames')

paths=['fpga_cpu_stats.txt', 'gpu_cpu_stats.txt', 'fpga_gpu_cpu_stats.txt']
read_files(paths, ['FPGA/CPU', 'GPU/CPU', 'FPGA/GPU/CPU'])